In [9]:
import os
import time
import smtplib
import mimetypes
from email.message import EmailMessage
from email.utils import formataddr
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from smtplib import SMTPException
from time import sleep

# Email configuration
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
USERNAME = 'nbhusana@cisco.com'
PASSWORD = 'Naveen@123456'
SENDER = 'nbhusana@cisco.com'
RECIPIENTS = ['nsbgoudra2001@gmail.com', 'ratnashirkol291283@gmail.com']
RETRY_LIMIT = 3  # Number of times to retry sending email in case of failure
RETRY_DELAY = 5  # Time (seconds) to wait before retrying

# Directory to monitor
MONITORED_DIR = '/path/to/monitor'

# Function to send email
def send_email(file_path):
    msg = EmailMessage()
    msg['From'] = formataddr(('Sender Name', SENDER))
    msg['To'] = ', '.join(RECIPIENTS)
    msg['Subject'] = f'New File Detected: {os.path.basename(file_path)}'

    # Attach file
    ctype, encoding = mimetypes.guess_type(file_path)
    if ctype is None or encoding is not None:
        ctype = 'application/octet-stream'
    
    maintype, subtype = ctype.split('/', 1)
    
    with open(file_path, 'rb') as fp:
        msg.add_attachment(fp.read(), maintype=maintype, subtype=subtype, filename=os.path.basename(file_path))

    # Send email with retries
    for attempt in range(RETRY_LIMIT):
        try:
            with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
                server.starttls()
                server.login(USERNAME, PASSWORD)
                server.send_message(msg)
            print(f"Email sent successfully with file: {file_path}")
            break
        except SMTPException as e:
            print(f"Failed to send email. Attempt {attempt + 1} of {RETRY_LIMIT}. Error: {e}")
            if attempt + 1 == RETRY_LIMIT:
                print(f"Max retries reached. Email could not be sent.")
            else:
                sleep(RETRY_DELAY)

# Event handler for file system events
class FileEventHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory:
            print(f"Detected new file: {event.src_path}")
            send_email(event.src_path)

# Start monitoring the directory
def start_monitoring():
    event_handler = FileEventHandler()
    observer = Observer()
    observer.schedule(event_handler, MONITORED_DIR, recursive=False)
    observer.start()
    print(f"Monitoring directory: {MONITORED_DIR}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

if __name__ == '_main_':
    start_monitoring()